# Landsat annual composites

1. fetch the Landsat scenes overlapping our study region
2. aggregate the Landsat scenes into a single multi-temporal composite using the median, our composite will contain the median reflectance obtained from all Landsat scenes fetched in certain year.

In [6]:
# we have to install the development version for the time being
# !pip uninstall eodal -y
# !pip install git+https://github.com/lukasValentin/eodal@landsat-dev

Found existing installation: eodal 0.2.1
Uninstalling eodal-0.2.1:
  Successfully uninstalled eodal-0.2.1
  Cloning https://github.com/lukasValentin/eodal (to revision landsat-dev) to c:\users\finnt\appdata\local\temp\pip-req-build-ub2icez8
  Resolved https://github.com/lukasValentin/eodal to commit 15deacfd6ed7a14e98ecda38414c7e93d2a6a7d0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for eodal: filename=eodal-0.2.1-py3-none-any.whl size=191302 sha256=81e0c8b6b1efe51ef0b37c125b206a36d08262df0723163c67aab2675da0ed25
  Stored in directory: C:\Users\finnt\AppData\Local\Temp\pip-ephem-wheel-cache-wsf5bi5y\wheels\2d\ef\47\5a8ebd8f2bc0aa9bc19e8e17cd1129e52d90ebeec080a79690
Successfully built eodal

  Running command git clone --filter=blob:none --quiet https://github.com/lukasValentin/eodal 'C:\Users\finnt\AppData\Local\Temp\pip-req-build-ub2icez8'
  Running command git checkout -b landsat-dev --track origin/landsat-dev
  branch 'landsat-dev' set up to track 'origin/landsat-dev'.
  Switched to a new branch 'landsat-dev'


In [ ]:
years = [1984, 1986, 1987, 1989, 1990, 1994, 1995, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [ ]:
from datetime import datetime
from pathlib import Path
from shapely.geometry import box
from matplotlib import pyplot as plt
import numpy as np

from eodal.config import get_settings
from eodal.core.sensors import Landsat
from eodal.mapper.feature import Feature
from eodal.mapper.filter import Filter
from eodal.mapper.mapper import Mapper, MapperConfigs
from eodal.core.band import GeoInfo, Band
from eodal.core.raster import RasterCollection

In [ ]:
# define a function to mask clouds and shadows
def preprocess_landsat_scene(
        ds: Landsat
) -> Landsat:
    """
    Mask clouds and cloud shadows in a Landsat scene based
    on the 'qa_pixel' band.

    NOTE:
        Depending on your needs, the pre-processing function can be
        fully customized using the full power of EOdal and its
        interfacing libraries!

    :param ds:
        Landsat scene before cloud mask applied.
    :return:
        Landsat scene with clouds and cloud shadows masked.
    """
    ds.mask_clouds_and_shadows(inplace=True)
    return ds

In [ ]:
## Setting up the EOdal Mapper
Settings = get_settings()

# we use STAC, i.e., Microsoft Planetary Computer
Settings.USE_STAC = True

# user-inputs
# -------------------------- Collection -------------------------------
collection = 'landsat-c2-l2'

# ---------------------- Spatial Feature  ------------------------------
# can be also shp, gpkg, etc.

# bbox = box(*[30.2825, 0.4019, 30.3714, 0.4643]) # 1. Area
# bbox = box(*[30.2703, 0.4043, 30.4482, 0.5291]) # 2. Area
# bbox = box(*[30.3195, 0.4803, 30.3417, 0.4959]) # 3. Area small for MeanShift trial
bbox = box(*[30.2433, 0.3746, 30.5640, 0.6962]) # 3. Area small for MeanShift trial

feature = Feature(
    name='landsat-composite',
    geometry=bbox,
    epsg=4326,
    attributes={})

# ------------------------- Metadata Filters ---------------------------
metadata_filters = [
    Filter('eo:cloud_cover', '<=', 30),
    #Filter('landsat:wrs_path', '==', '173'),
    #Filter('landsat:wrs_row', '==', '060'),
    #Filter('instruments', '!=', 'etm+')
]

for year in years: 
    
    # ------------------------- Time Range ---------------------------------
    time_start = datetime(year, 1, 1)
    time_end = datetime(year, 12, 31)

    # set up the Mapper configuration
    mapper_configs = MapperConfigs(
        metadata_filters=metadata_filters,
        collection=collection,
        feature=feature,
        time_start=time_start,
        time_end=time_end)

    # get a new mapper instance
    mapper = Mapper(mapper_configs)

    # fetch the metadata
    # query the scenes available (no I/O of scenes, this only fetches metadata)
    mapper.query_scenes()
    print(str(year), f'Number of Landsat scenes found: {mapper.metadata.shape[0]}')

    # define e bands to read in 'band_selection' and to preprocess the scenes
    scene_kwargs = {
        'scene_constructor': Landsat.from_usgs,
        'scene_constructor_kwargs': {
            'band_selection': ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'qa_pixel'], 
            'read_qa': False} ,
        'scene_modifier': preprocess_landsat_scene,
        'scene_modifier_kwargs': {}}

    # now we load the scenes
    mapper.load_scenes(scene_kwargs=scene_kwargs)

    # The mapper returns the single scenes. 
    # As we told the EOdal to mask out clouds, a significant share of the pixels is masked out. 
    # We will aggregate them in the next step.
    f = mapper.data.plot(band_selection=['red', 'green', 'blue'], figsize = (20, 20), max_scenes_in_row = 2)

    #save as PNG for quick view
    f.savefig('S:\MSc_23_TimckeFinn\data\python_outputs\landsat_scenes_' + str(year) + '.png')
    
    #First, we open masked arrays for storing the data:
    # all scenes have the same shape, i.e., the same number of bands, rows and columns
    shapes = [{timestamp: scene.get_values().shape} for timestamp, scene in mapper.data]

    # open arrays for storing the data per band
    shape = (len(mapper.data), list(shapes[0].values())[0][1], list(shapes[0].values())[0][2])
    blue = np.ma.masked_array(data=np.ndarray(shape, dtype=float), mask=False)
    red = np.zeros_like(blue)
    green = np.zeros_like(blue)
    nir08 = np.zeros_like(blue)
    swir16 = np.zeros_like(blue)

    # Next, we loop over the scenes.
    idx = 0
    for _, scene in mapper.data:
        blue[idx, :, :] = scene['blue'].values
        red[idx, :, :] = scene['red'].values
        green[idx, :, :] = scene['green'].values
        nir08[idx, :, :] = scene['nir08'].values
        swir16[idx, :, :] = scene['swir16'].values
        idx += 1

    # Finally, we aggregate the data using the median reflectance
    # calculate the median reflectance per spectral band
    blue_median = np.ma.median(blue, axis=0)
    green_median = np.ma.median(green, axis=0)
    red_median = np.ma.median(red, axis=0)
    nir08_median = np.ma.median(nir08, axis=0)
    swir16_median = np.ma.median(swir16, axis=0)

    rc = RasterCollection()

    bands = {'blue': blue_median, 
            'green': green_median, 
            'red': red_median, 
            'nir08' : nir08_median, 
            'swir16' : swir16_median}

    for band_name, band_value in bands.items():
        rc.add_band(
            band_constructor=Band,
            values=band_value,
            band_name=f'{band_name}_median',
            geo_info=scene[band_name].geo_info)
        
    # # We plot the result
    f, ax = plt.subplots(figsize = (20, 10))
    f = rc.plot_multiple_bands(['red_median', 'green_median', 'blue_median'], ax=ax)

    #save as PNG for quick view
    f.savefig('S:\MSc_23_TimckeFinn\data\python_outputs\landsat_median_composite_' + str(year) + '.png')
        
    # save as GeoTiff for further analysis
    rc.to_rasterio('S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_' + str(year) + '.tif')
